In [1]:
import easyocr
import fitz  # PyMuPDF
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from groq import Groq
import os
from dotenv import load_dotenv


from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader


from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_ollama.llms import OllamaLLM
from typing import Optional

In [2]:
# Define the paths for the directories
directories = [
    "../outputs/extracted_images",
    "../outputs/extracted_text",
    "../outputs/extracted_products"
]

# Create the directories if they do not exist
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")

Directory already exists: ../outputs/extracted_images
Directory already exists: ../outputs/extracted_text
Directory already exists: ../outputs/extracted_products


In [10]:

load_dotenv()

## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY']
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


In [ ]:
def convert_pdf_to_images(pdf_path, output_folder):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Iterate through each page
    for page_num in range(len(pdf_document)):
        # Get the page
        page = pdf_document.load_page(page_num)
        
        # Get the page's pixmap (image)
        pix = page.get_pixmap()
        
        # Convert the pixmap to a PIL Image
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Save the image
        image_path = f"{output_folder}/page_{page_num + 1}.png"
        image.save(image_path)
        print(f"Saved page {page_num + 1} as {image_path}")

    print("All pages converted to images.")

In [7]:
def overlay_extracted_text_on_image(image_path, extract_info):

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Iterate through the extracted information
    for el in extract_info:
        # el[0] contains the bounding box coordinates
        # el[1] contains the text
        # el[2] contains the confidence score
        bbox = el[0]
        text = el[1]
        confidence = el[2]
        
        # Convert the coordinates from list of tuples to integer
        bbox = [(int(x), int(y)) for (x, y) in bbox]
        
        # Draw the bounding box on the image
        cv2.polylines(image, [np.array(bbox)], isClosed=True, color=(0, 255, 0), thickness=2)
        
        # Draw the text on the image
        cv2.putText(image, f"{text} ({confidence:.2f})", (bbox[0][0], bbox[0][1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Convert image from BGR to RGB for display with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image with bounding boxes and text using matplotlib
    plt.figure(figsize=(30, 15))  # Adjust size as needed
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide axes
    plt.show()

In [ ]:
def extract_text_from_image(image_path, extracted_text_path, lang = "fr"):
    reader = easyocr.Reader([lang])
    extract_info = reader.readtext(image_path)

    extracted_text = ""
    for el in extract_info:
        extracted_text += el[1] + "\n"

    # Save the text to a file
    with open(extracted_text_path, 'w') as file:
        file.write(extracted_text)

## Extract pages from flyer and run OCR on each page

In [ ]:
pdf_path = '../data/flyer.pdf'
extracted_images_folder = '../outputs/extracted_images'
convert_pdf_to_images(pdf_path, extracted_images_folder)

In [ ]:
for page_num in range(26):
    image_path = f"../outputs/extracted_images/page_{page_num + 1}.png"
    extracted_text_path = f"../outputs/extracted_text/page_e_{page_num + 1}.txt"
    extract_text_from_image(image_path, extracted_text_path, 'en')

In [20]:
content= "Here\u2019s the extracted information from the flyer, organized by product:\n\n### 1. Sun-Maid Raisin Bread\n- **Product Name/Brand**: Sun-Maid Raisin Bread\n- **Price**: Not specified\n- **Promotions**: 2 pour 1 (Buy one, get one free)\n\n### 2. Tortillas\n- **Product Name/Brand**: Tortillas (Assorted selection)\n- **Price**: $2.99\n- **Promotions**: None listed\n\n### 3. Oikos Yogurt\n- **Product Name/Brand**: Oikos\n- **Price**: $6.99\n- **Promotions**: None listed\n\n### 4. Pearl Milling Company Pancake Mix\n- **Product Name/Brand**: Pearl Milling Company\n- **Price**: $3.99\n- **Promotions**: None listed\n\n### 5. Quaker Instant Grits\n- **Product Name/Brand**: Quaker\n- **Price**: $3.49\n- **Promotions**: None listed\n\n### 6. Folgers Coffee\n- **Product Name/Brand**: Folgers\n- **Price**: $17.99\n- **Promotions**: None listed\n\n### 7. Philadelphia Cream Cheese\n- **Product Name/Brand**: Philadelphia\n- **Price**: $6.99\n- **Promotions**: None listed\n\n### 8. Kashi GoLean Cereal\n- **Product Name/Brand**: Kashi\n- **Price**: $4.99\n- **Promotions**: None listed\n\n### 9. Tim Hortons Hot Chocolate\n- **Product Name/Brand**: Tim Hortons\n- **Price**: $10.99\n- **Promotions**: None listed\n\n### 10. Frozen Belgian Waffles\n- **Product Name/Brand**: Belgium Waffles\n- **Price**: $4.49\n- **Promotions**: 50 PTS (points)\n\n### 11. Pure Leaf Tea\n- **Product Name/Brand**: Pure Leaf\n- **Price**: Not specified\n- **Promotions**: None listed\n\n### 12. Lactantia Milk (Lactose-Free)\n- **Product Name/Brand**: Lactantia\n- **Price**: $4.99\n- **Promotions**: None listed\n\nIf there are additional products or specific details you'd like to include or explore further, let me know!"
for text in content.split("###"):
    print(text)

extracted_text_path = f"../outputs/flyer.txt"
# Save the text to a file
with open(extracted_text_path, 'w') as file:
    file.write(content)


Here’s the extracted information from the flyer, organized by product:


 1. Sun-Maid Raisin Bread
- **Product Name/Brand**: Sun-Maid Raisin Bread
- **Price**: Not specified
- **Promotions**: 2 pour 1 (Buy one, get one free)


 2. Tortillas
- **Product Name/Brand**: Tortillas (Assorted selection)
- **Price**: $2.99
- **Promotions**: None listed


 3. Oikos Yogurt
- **Product Name/Brand**: Oikos
- **Price**: $6.99
- **Promotions**: None listed


 4. Pearl Milling Company Pancake Mix
- **Product Name/Brand**: Pearl Milling Company
- **Price**: $3.99
- **Promotions**: None listed


 5. Quaker Instant Grits
- **Product Name/Brand**: Quaker
- **Price**: $3.49
- **Promotions**: None listed


 6. Folgers Coffee
- **Product Name/Brand**: Folgers
- **Price**: $17.99
- **Promotions**: None listed


 7. Philadelphia Cream Cheese
- **Product Name/Brand**: Philadelphia
- **Price**: $6.99
- **Promotions**: None listed


 8. Kashi GoLean Cereal
- **Product Name/Brand**: Kashi
- **Price**: $4.99
- **P

## Extract product using local model or runing on Groq

In [4]:
flyer_content = []
for page_num in range(26):
    file_path = f"../outputs/extracted_text/page_e_{page_num + 1}.txt"
    with open(file_path, 'r') as file:
        flyer_content.append(file.read())

In [5]:
len(flyer_content)


26

In [ ]:
model_local = OllamaLLM(model="llama3",
                   temperature=0.0)

model_groq=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.1-70b-versatile",
             temperature=0.0)


# Define your desired data structure.
class ExtractedProducts(BaseModel):
    food_products: list[str] = Field(description="List of all the food products in English.")
    other_products: Optional[list[str]] = Field(description="List of any products other than food products in English.")
    reasonings: Optional[str] = Field(description="Reasoning for the categorization.", default=None)


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ExtractedProducts)


template = """Your task is to analyze the text and extract food items and other products mentioned in English.
Translate any non-English items to English. Only include items that can be used for cooking or consumption in the 'food_products' list. Exclude any non-food items like cleaning products, clothes, or any other items that cannot be used for cooking.

**Food products** are items that are edible or can be used in preparing meals. Examples include:
- Fruits and vegetables (e.g., 'Apple', 'Carrot')
- Dairy products (e.g., 'Milk', 'Cheese')
- Meat and poultry (e.g., 'Chicken', 'Beef')
- Grains and cereals (e.g., 'Rice', 'Oats')

**Non-food products** include items that are not used for cooking or consumption. Examples include:
- Cleaning products (e.g., 'Detergent', 'Cleaner')
- Clothing (e.g., 'Shirt', 'Jacket')
- Miscellaneous non-food items (e.g., 'Packaging', 'Discount Coupons')

Provide your response in the following format:

{format_instructions}

Text to analyze:
{query}
"""


prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | model_groq

for i in range(0,len(flyer_content)):
    print(f"processing page_{i + 1}")
    extracted_products_path = f"../outputs/extracted_products/page_{i + 1}.txt"
    output = prompt_and_model.invoke({"query": flyer_content[i]})
    extracted_product_p = parser.invoke(output)
    print(extracted_product_p)

    # Save the text to a file
    with open(extracted_products_path, 'w') as file:
        file.write(", ".join(extracted_product_p.food_products))


In [6]:

extracted_product = []
for page_num in range(26):
    file_path = f"../outputs/extracted_products/page_{page_num + 1}.txt"
    with open(file_path, 'r') as file:
        extracted_product.append(file.read())
        
for i in range(len(extracted_product)):
  print("*"*20, "page ", i+1, "*"*20)
  print(extracted_product[i])


******************** page  1 ********************
Ketchup, Tomato Ketchup, Gelato, Ice Cream, Pork Back Ribs, Veal, Pork, Natural Spring Water, Pepsi, Coca-Cola, Hamburger, Hot Dog, Sausages, Bagels, Muffins, Iceberg Lettuce, Broccoli, Blueberries, Corona Beer, Sapporo Beer
******************** page  2 ********************
BACON, SAUCISSES FRAICHES, TOMATES ROUGES DE SERRE, GREENHOUSE RED TOMATOES, WHITE EGGS, PORK & BEEF, YOGOURT, SKYR, CEREALES, JUS DE FRUITS, FRUIT JUICE, NECTAR DE FRUITS, PIZZA, PAIN TRANCHE, REPS, GOURMET STEAMERS, PURE BREAKFAST, FROZEN THIN CRUST PIZZA, SLICED BREAD, MUFFIN, CORN, CARRES DE RIZ, TARTINADE, NUTELLA, FRUIT-O-LONG, FRUITSATIONS, RICE, TORTILLAS, BOISSON, ALCOOLISEE, BUD LIGHT CHELADA, WHITE CLAW, ALCOHOLIC MALT, BEVERAGE, CLAMATO
******************** page  3 ********************
Beer, Watermelon, Cherries, Ice Cream, Frozen Yogurt, Gnocchi, Pasta Sauce, Pesto, Muffins, Skillet, Red Cherries, Melon, Vanille, Cerises Rouges, Bud Light, Molson Ultra, 

In [7]:
', '.join(extracted_product)

"Ketchup, Tomato Ketchup, Gelato, Ice Cream, Pork Back Ribs, Veal, Pork, Natural Spring Water, Pepsi, Coca-Cola, Hamburger, Hot Dog, Sausages, Bagels, Muffins, Iceberg Lettuce, Broccoli, Blueberries, Corona Beer, Sapporo Beer, BACON, SAUCISSES FRAICHES, TOMATES ROUGES DE SERRE, GREENHOUSE RED TOMATOES, WHITE EGGS, PORK & BEEF, YOGOURT, SKYR, CEREALES, JUS DE FRUITS, FRUIT JUICE, NECTAR DE FRUITS, PIZZA, PAIN TRANCHE, REPS, GOURMET STEAMERS, PURE BREAKFAST, FROZEN THIN CRUST PIZZA, SLICED BREAD, MUFFIN, CORN, CARRES DE RIZ, TARTINADE, NUTELLA, FRUIT-O-LONG, FRUITSATIONS, RICE, TORTILLAS, BOISSON, ALCOOLISEE, BUD LIGHT CHELADA, WHITE CLAW, ALCOHOLIC MALT, BEVERAGE, CLAMATO, Beer, Watermelon, Cherries, Ice Cream, Frozen Yogurt, Gnocchi, Pasta Sauce, Pesto, Muffins, Skillet, Red Cherries, Melon, Vanille, Cerises Rouges, Bud Light, Molson Ultra, Bud, Sleeman Clear 2.0, Miss Vickie's, Olivieri, Budweiser, Mushrooms, Mandarins, Portabella, Ranch dressing, Pork, Ground pork, Barley, Chicken, C

# categorize in geredient

In [13]:
model_local = OllamaLLM(model="llama3",
                   temperature=0.0)

model_groq=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.1-70b-versatile",
             temperature=0.0)


class CategorizeIngredients(BaseModel):
    meat: Optional[list[str]] = Field(default=None, description="List of meats.")
    fruits: Optional[list[str]] = Field(description="List of fruits.")
    vegetables: Optional[list[str]] = Field(default=None, description="List of vegetables.")
    dairy: Optional[list[str]] = Field(default=None, description="List of dairy products.")
    grains_pasta: Optional[list[str]] = Field(default=None, description="List of grains and pasta.")
    canned_prepared_foods: Optional[list[str]] = Field(default=None, description="List of canned and prepared foods.")
    condiments_sauces: Optional[list[str]] = Field(default=None, description="List of condiments and sauces.")
    beverages: Optional[list[str]] = Field(default=None, description="List of beverages.")
    others: Optional[list[str]] = Field(default=None, description="List of other ingredients.")
    reasonings: Optional[str] = Field(default=None, description="Reasoning for the categorization.")


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=CategorizeIngredients)



template = """
Your task is to categorize the ingredients provided into the appropriate categories.
Food products are items that are edible or can be used in preparing meals. Here are the categories and some examples:

- **Fruits**: Nectarines, Peaches, Apricots, Plums, Avocados
- **Meats**: Chicken, Poulet, Chicken Noodle, Tacos
- **Vegetables**: Vegetables, Mixed Legumes, Cream-style Corn
- **Dairy**: Sour Cream, Cottage Cheese, Mozzarella, Babybel, Refrigerated Pudding
- **Grains & Pasta**: Spaghettini, Pasta, Lasagna, Nouilles
- **Canned & Prepared Foods**: Compote de Fruits, Compote, Rice Pudding, Soup, Frozen Meal, Meal Kit
- **Condiments & Sauces**: Mayonnaise, Salsa
- **Beverages**: Jus, Boisson, Cocktail aux Fruits, Thé Glacé

If you cannot categorize an ingredient into any of the provided categories, add it to the 'Others' category.

Provide your response as a valid JSON object in the following format:

{format_instructions}

Ingredients to categorize:
{query}
"""


prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | model_groq

output = prompt_and_model.invoke({"query": ', '.join(extracted_product)})
categorize_ingredients = parser.invoke(output)
print(categorize_ingredients)

meat=['Pork Back Ribs', 'Veal', 'Pork', 'Hamburger', 'Hot Dog', 'Sausages', 'BACON', 'SAUCISSES FRAICHES', 'PORK & BEEF', 'Beef', 'Chicken', 'Ground pork', 'Bratwurst', 'Smoked sausages', 'Country-style pate', 'Liver pate', 'Smoked sausages', 'Beef', 'Duck Confit', 'Shredded Duck', 'Strip Loin Steak', 'Bacon', 'Pork', 'Pork Shanks', 'Smoked Ham', 'Prosciutto', 'Salami', 'Genoa Salami', 'Sliced Salami', 'Dry Sausages', 'Cretons', 'Old-Fashioned Cretons', 'Lamb', 'Lamb Loin Chops', 'Chicken Legs', 'Chicken Thighs', 'Chicken Drumsticks', 'Chicken', 'Pork', 'Fish', 'Salmon', 'Shrimp', 'Scallops', 'Lamb cutlets', 'Poulet', 'General tao', 'Chicken wings', 'Chicken breast fillets', 'Chicken chunks', 'Lamb', 'Beef', 'Lasagna', 'Chicken nuggets', 'Frankfurters', 'Drumsticks', 'Tenders', 'Ham', 'Salami', 'Bologna', 'Pepperoni', 'Hot Dog', 'Smoked Meat', 'Bacon', 'Smoked Bacon', 'Smoked Salmon', 'Maple Salmon', 'Beef Salami', 'Wiener', 'Sliced Smoked Meat', 'Frozen Smoked Salmon', 'Turkey', 'Grou

In [ ]:
import json

def safe_parse_json(json_string):
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        # Attempt to fix common errors or handle it gracefully
        print("Error parsing JSON:", e)
        # Optionally try to fix issues here
        return None

parsed_output = safe_parse_json(output)
print(parsed_output)

In [14]:
print("Meat", categorize_ingredients.meat),
print("Fruits", categorize_ingredients.fruits),
print("Vegetables", categorize_ingredients.vegetables),
print("Dairy", categorize_ingredients.dairy),
print("Grains & Pasta", categorize_ingredients.grains_pasta),
print("Canned & Prepared Foods", categorize_ingredients.canned_prepared_foods),
print("Condiments & Sauces", categorize_ingredients.condiments_sauces),
print("Beverages", categorize_ingredients.beverages),
print("Others", categorize_ingredients.others),
print("Reasonings", categorize_ingredients.reasonings)

Meat ['Pork Back Ribs', 'Veal', 'Pork', 'Hamburger', 'Hot Dog', 'Sausages', 'BACON', 'SAUCISSES FRAICHES', 'PORK & BEEF', 'Beef', 'Chicken', 'Ground pork', 'Bratwurst', 'Smoked sausages', 'Country-style pate', 'Liver pate', 'Smoked sausages', 'Beef', 'Duck Confit', 'Shredded Duck', 'Strip Loin Steak', 'Bacon', 'Pork', 'Pork Shanks', 'Smoked Ham', 'Prosciutto', 'Salami', 'Genoa Salami', 'Sliced Salami', 'Dry Sausages', 'Cretons', 'Old-Fashioned Cretons', 'Lamb', 'Lamb Loin Chops', 'Chicken Legs', 'Chicken Thighs', 'Chicken Drumsticks', 'Chicken', 'Pork', 'Fish', 'Salmon', 'Shrimp', 'Scallops', 'Lamb cutlets', 'Poulet', 'General tao', 'Chicken wings', 'Chicken breast fillets', 'Chicken chunks', 'Lamb', 'Beef', 'Lasagna', 'Chicken nuggets', 'Frankfurters', 'Drumsticks', 'Tenders', 'Ham', 'Salami', 'Bologna', 'Pepperoni', 'Hot Dog', 'Smoked Meat', 'Bacon', 'Smoked Bacon', 'Smoked Salmon', 'Maple Salmon', 'Beef Salami', 'Wiener', 'Sliced Smoked Meat', 'Frozen Smoked Salmon', 'Turkey', 'Grou

## Recommand recepies using Groq

In [18]:


file_path = ( "../data/recipes/raw_unique_recipes.csv")

loader = CSVLoader(file_path=file_path)
recipes_data = loader.load()



In [16]:
len(recipes_data)

230489

In [19]:
for record in recipes_data[:200]:
    print()
    print("*"*20)
    print(record)


********************
page_content='name: arriba   baked winter squash mexican style
preparation_time: 55
directions: ['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']
ingredients: ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
calories: 51.5
total fat (PDV): 0.0
sugar (

In [20]:
#embeddings=OllamaEmbeddings(model="llama3")
embeddings=OpenAIEmbeddings()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) 
final_documents=text_splitter.split_documents(recipes_data[:200])
vectors=FAISS.from_documents(final_documents,embeddings) 


/home/shima/projects/budget_meal_planner/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [21]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.1-70b-versatile",
             temperature=0.0)




prompt = ChatPromptTemplate.from_template(
"""
Your job is to find a recipe from the provided context that use the given ingredients. 
Ensure that each recipe is described with the following fields:
- **Recipe 1: 
- **name**: The name of the recipe.
- **preparation_time**: The time required to prepare the recipe.
- **directions**: A list of instructions for preparing the recipe.
- **ingredients**: A list of ingredients required for the recipe.
- **calories**: The total number of calories in the recipe.
- **total fat (PDV)**: Percentage of daily value for total fat.
- **sugar (PDV)**: Percentage of daily value for sugar.
- **sodium (PDV)**: Percentage of daily value for sodium.
- **protein (PDV)**: Percentage of daily value for protein.
- **saturated fat (PDV)**: Percentage of daily value for saturated fat.
- **carbohydrates (PDV)**: Percentage of daily value for carbohydrates.

The recipes must be selected from the context provided below. If any ingredients are missing from the list, include them in the recipe details.

If you cannot find a recipe that meets the criteria, please state that you don’t know.

<contex>
{context}
</context>

Questions:
{input}
"""
)





In [22]:

user_message="Find a recipe that includes chicken breast as an ingredient and has less than 200 calories per serving."


document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectors.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
response=retrieval_chain.invoke({'input':user_message})

In [ ]:
for key, value in response.items():
    print()
    print("*"*20)
    print(key)
   # print(value)

In [23]:
response["context"]

[Document(metadata={'source': '../data/recipes/raw_unique_recipes.csv', 'row': 164}, page_content="ingredients: ['chicken breasts', 'baby carrots', 'celery', 'white onion', 'chicken stock', 'kosher salt', 'pepper', 'matzo meal', 'eggs', 'oil', 'salt', 'sugar', 'baking powder']\ncalories: 361.1\ntotal fat (PDV): 34.0\nsugar (PDV): 25.0\nsodium (PDV): 39.0\nprotein (PDV): 45.0\nsaturated fat (PDV): 23.0\ncarbohydrates (PDV): 5.0"),
 Document(metadata={'source': '../data/recipes/raw_unique_recipes.csv', 'row': 84}, page_content="ingredients: ['boneless skinless chicken thighs', 'all-purpose flour', 'salt', 'fresh ground black pepper', 'dried oregano', 'dried thyme', 'garlic powder', 'vegetable oil', 'onion', 'chicken stock', 'dry white wine', 'breadcrumbs', 'milk', 'lean ground beef', 'poultry seasoning']\ncalories: 692.2\ntotal fat (PDV): 53.0\nsugar (PDV): 16.0\nsodium (PDV): 17.0\nprotein (PDV): 102.0\nsaturated fat (PDV): 36.0\ncarbohydrates (PDV): 12.0"),
 Document(metadata={'source'

In [24]:
for answer in response["answer"].split("\n\n"):
    print(answer)

**Recipe 1:**
- **name**: Steamed Chicken Cutlets in Packages
- **preparation_time**: 35 minutes
- **directions**: 
  1. Preheat oven to 450F.
  2. Rinse the chicken breasts, and either cut or pound the chicken to create cutlets of about equal size.
  3. Tear off squares of aluminum foil and place each chicken breast on a square.
  4. Slice the tomato and place a slice or two atop of each breast.
  5. Drizzle the chicken with the olive oil.
  6. Sprinkle the basil on top of the chicken breasts evenly.
  7. Add salt and pepper to each breast.
  8. Loosely fold up the foil over each cutlet, leaving both room for air space but folded up enough not to allow oil to drip out.
  9. Place all packages in a large baking dish and bake for 20 minutes.
  10. Serve while still packaged.
  11. Allow each diner to open his/her package at the table.
- **ingredients**: 
  - boneless skinless chicken breast
  - tomatoes
  - dried basil
  - olive oil
  - lime juice
  - salt and pepper
- **calories**: 190

In [ ]:
for doc_source in response["context"]:
    print(doc_source)